In [1]:

class DaySet:
    def __init__(self,day,date,day_limit):
        self.day = day
        self.day_limit = day_limit
        self.date = date
        self.attractions = []

class Timing:
    def __init__(self,day,open,st_time,en_time):
        self.day = day
        self.open = open
        self.st_time = st_time
        self.en_time = en_time

class Attraction:
    def __init__(self,id:int,duration:int,timings:list,rating:float,location:list,at_type:int):
        self.id = id
        self.duration = duration
        self.rating = rating
        self.location = location
        self.type = at_type
        self.timings = timings
    
    # Define comparison for less than
    def __lt__(self, other):
        return self.rating > other.rating

        

In [ ]:
from geopy.distance import geodesic

def time_between_locations(l1:list,l2:list):
    distance = geodesic(l1, l2).kilometers
    travel_time = distance / 30  # Assuming average speed of 30 km/h
    return travel_time

def get_timing(day_set:DaySet, attraction:Attraction):
    if len(day_set.attractions) == 0:
        return 0
    
    best_time = float("inf")
    for selected_attraction in day_set.attractions:
        travel_time = time_between_locations(selected_attraction.location, attraction.location)
        best_time = min(best_time,travel_time)
    return best_time

def average_rating(attractions:list):
    total_rating = 0
    total_duration = 0
    for attraction in attractions:
        total_rating += attraction.rating
        total_duration += attraction.duration
    return total_rating/total_duration

def get_effective_rating(day_set:DaySet, attraction:Attraction,time_penelty_factor:float):
    LENIENT_TIMING = 0.5
    effective_timing = max(get_timing(day_set,attraction) - LENIENT_TIMING,0)
    effective_rating = attraction.rating - effective_timing*time_penelty_factor

def get_rating_matrix(day_sets:list,attractions:list): 
    AVG_RATING = average_rating(attractions)
    rating = []
    for i in range(len(day_sets)):
        for j in range(len(attractions)):
                cur_day = day_sets[i].day
                new_day_set = day_sets[i].attractions + [attractions[j]]
                if day_sets[i].day != attraction[j].timings.day or get_day_iternery(converted_attractions(new_day_set))==None: #attraction is closed on that day or not possible
                    rating.append([i,j,0])
                else:
                    effective_rating = get_effective_rating(day_sets[i],attractions[j])
                    rating.append([i,j,effective_rating])
    rating.sort(key=lambda x: x[2])
    return rating



In [2]:
import sqlite3

MAX_RATING = 40
STD_TIMING = [
    (-1, -1, 0, 1, '1000', '1800'), 
    (-1, -1, 1, 1, '1000', '1800'), 
    (-1, -1, 2, 1, '1000', '1800'), 
    (-1, -1, 3, 1, '1000', '1800'), 
    (-1, -1, 4, 1, '1000', '1800'), 
    (-1, -1, 5, 1, '1000', '1800'), 
    (-1, -1, 6, 1, '1000', '1800')
    ]

ALL_TIMING = [
    (-1, -1, 0, 1, '0800', '2400'), 
    (-1, -1, 1, 1, '0800', '2400'), 
    (-1, -1, 2, 1, '0800', '2400'), 
    (-1, -1, 3, 1, '0800', '2400'), 
    (-1, -1, 4, 1, '0800', '2400'), 
    (-1, -1, 5, 1, '0800', '2400'), 
    (-1, -1, 6, 1, '0800', '2400')
    ]


def open247(day_results):
    for day in day_results:
        if day[3] == 1:
            return 0
    return 1

def get_attraction_list(requirement):
    types_string = ",".join(["'"+str(type_id)+"'" for type_id in requirement["types"]])
    query = f"""
    SELECT * FROM Attractions
    WHERE city_id = (SELECT id FROM Cities WHERE name = '{requirement["city"]}') 
    AND type_id IN (SELECT id FROM Types WHERE name IN ({types_string}))
    """
    conn = sqlite3.connect("../Data/tws.db")
    cursor = conn.cursor()
    cursor.execute(query)
    attractions_results = cursor.fetchall()


    attractions = []
    for row in attractions_results:
        id, duration, rating, location, at_type = row[0],row[5],row[6],[row[8],row[9]],row[2]
        type_query = f"""
            SELECT name from Types
            WHERE id = {at_type}
        """
        cursor.execute(type_query)
        at_type = cursor.fetchall()[0][0]
        if id in requirement["exclude"]:
            continue
        if id in requirement["include"]:
            rating = MAX_RATING
        if id in requirement["duration"][0]:
            rating = MAX_RATING
            duration = requirement["duration"][1][requirement["duration"][0].index(id)]    
    
        time_query = f"""
            SELECT * FROM OperationHours
            WHERE attraction_id = {id}
        """     
        cursor.execute(time_query)
        day_results = cursor.fetchall()
        if len(day_results)==1:
            day_results = STD_TIMING
        if open247(day_results):
            day_results = ALL_TIMING
            
        timings=[]
        for i in range(0,7):
            timings.append(Timing(
                i,
                int(day_results[i][3]),
                int(day_results[i][4]) if day_results[i][4] is not None else -1,
                int(day_results[i][5]) if day_results[i][5] is not None else -1
            ))
        attractions.append(Attraction(id,duration,timings,rating,location,at_type))
    
    return(attractions)


requirement = {
    "city" : "London",
    "st_date" : "03-08-23",
    "en_date" : "04-08-23",
    "types" : ["Historical & Cultural","Food & Drinks"],
    "exclude" : [2],
    "include" : [3],
    "duration" : [[4,5],[2,6]],
}

for ele in get_attraction_list(requirement):
    print(ele.__dict__)
    for tm in ele.__dict__['timings']:
        print(tm.__dict__)

{'id': 3, 'duration': 3, 'rating': 40, 'location': [51.50811239999999, -0.0759493], 'type': 'Historical & Cultural', 'timings': [<__main__.Timing object at 0x105de4070>, <__main__.Timing object at 0x105de4100>, <__main__.Timing object at 0x105d776d0>, <__main__.Timing object at 0x105c6a4f0>, <__main__.Timing object at 0x105c6a070>, <__main__.Timing object at 0x105c6a7f0>, <__main__.Timing object at 0x105dd6f10>]}
{'day': 0, 'open': 1, 'st_time': 900, 'en_time': 1730}
{'day': 1, 'open': 1, 'st_time': 900, 'en_time': 1730}
{'day': 2, 'open': 1, 'st_time': 900, 'en_time': 1730}
{'day': 3, 'open': 1, 'st_time': 900, 'en_time': 1730}
{'day': 4, 'open': 1, 'st_time': 900, 'en_time': 1730}
{'day': 5, 'open': 1, 'st_time': 900, 'en_time': 1730}
{'day': 6, 'open': 1, 'st_time': 900, 'en_time': 1730}
{'id': 4, 'duration': 2, 'rating': 40, 'location': [51.5021585, -0.1293572], 'type': 'Historical & Cultural', 'timings': [<__main__.Timing object at 0x105dd68e0>, <__main__.Timing object at 0x105dc9

requirement : {
    city : string
    st_date : ?~?
    en_date : ?~?
    types : {} // type1, type2, type3 ...
    exclude : {} // id_1, id_2, id_3 ...
    include : {
        {},{},{} // {id_1,st_time, en_time}
    }
    duration : {
        
    }
}

request : {
    city : //From Index Page
    st_date : //From Index Page
    en_date : //From Index Page
    types : {} //From Attraction Page
    exclude : [] //Empty
    include : [] //Empty
    duration : [[],[]] //Empty
}

itinerary : {
    day1 : {
        
    }
}



## Day Iternery
`Input = [[opening_time,closing_time,duration,[loaction]]]`

In [ ]:
import itertools

def converted_attractions(attractions, day):

  converted_attractions = []

  for attraction in attractions:
    converted_attraction = {
            "id" : attraction.id,
          "opening_time": int(attraction.timings[day].st_time),
          "closing_time": int(attraction.timings[day].en_time),
          "duration": attraction.duration,
          "location": attraction.location
      }
    converted_attractions.append(converted_attraction)

  return converted_attractions


def get_day_iternery(locations):
    # Initialize a list to store the feasible orders of the locations.
    feasible_orders = []

    # Iterate over all combinations of the locations.
    for order in itertools.permutations(locations):
        cur_time = order[0]['opening_time'] + order[0]['duration']
        order_schedule = []  # Store the current order's schedule

        for i in range(1, len(order)):
            cur_time += time_between_locations(order[i - 1]['location'], order[i]['location'])
            cur_time = max(order[i]['opening_time'], cur_time)
            exit_time = cur_time + order[i]['duration']

            if exit_time >= order[i]['closing_time']:
                break

            # Append the id, start time, and end time to the order_schedule
            order_schedule.append([order[i]['id'], cur_time, exit_time])
            cur_time = exit_time

        feasible_orders.append(order_schedule)

    # If there are no feasible orders, return None.
    if not feasible_orders:
        return None
    else:
        return feasible_orders[0]

# Example data
locations = [
    {'id': 1, 'opening_time': 9, 'closing_time': 18, 'duration': 2, 'location': 'A'},
    {'id': 2, 'opening_time': 10, 'closing_time': 18, 'duration': 3, 'location': 'B'},
    {'id': 3, 'opening_time': 11, 'closing_time': 18, 'duration': 2, 'location': 'C'}
]

result = get_day_iternery(locations)
print(result)


In [ ]:
DAY_LIMIT = 6
import heapq
from datetime import datetime

def itenery_scheduler(request):
    print(request)
    request['st_date'] = datetime.strptime(request['st_date'], "%Y-%m-%d").date()
    request['en_date'] = datetime.strptime(request['en_date'], "%Y-%m-%d").date()

    num_types = len(request['types'])
    num_days,start_day =  (request['en_date'] - request['st_date']).days, request['st_date'].weekday()
    daySets = []
    curDate,curDay = request['st_date'],start_day
    for i in range(num_days):
        new_day = DaySet(curDay,curDate,DAY_LIMIT)
        daySets.append(new_day)
    
    type_heaps = {}
    for types in request['types']:
        type_heaps[types] = []
    
    attractions = get_attraction_list(request)

    for attraction in attractions:
        type_heaps[attraction.type].append(attraction)
    
    for types in request['types']:
        heapq.heapify(type_heaps[types])

    typeSets = []

    for types in request['types']:
        typeSets.append(heapq.heappop(type_heaps['types']))    
    
    while(len(daySets)!=0 or len(typeSets)!=0):
        get_rating_matrix(daySets,typeSets)
    
    
    
request = {'city': 'Mumbai', 'st_date': '2023-08-10', 'en_date': '2023-08-11', 'types': ['Historical & Cultural', 'Amusement & Entertainment'], 'exclude': [], 'include': [], 'duration': [[], []]}
itenery_scheduler(request)


In [3]:
import sqlite3
def get_row(id):
    query = f"""
        SELECT * from Attractions WHERE id = {id}
    """
    conn = sqlite3.connect("../Data/tws.db")
    cursor = conn.cursor()
    cursor.execute(query)
    attractions_results = cursor.fetchall()[0]
    return attractions_results

get_row(934)

(934,
 0,
 1,
 'Haji Ali Mosque',
 'Sitting on a jetty in the Arabian sea, this spectacular white mosque was built to honor and house the remains of the Muslim saint Haji Ali.',
 2,
 12.79,
 'ChIJ40INk37O5zsRBY0Hq8dd4NU',
 18.9827269,
 72.8089112)

In [15]:
def add_time(t1,t2):
    h1,m1 = t1//100,t1%100
    m1+=t2
    h2,m2 = m1//60, m1%60

    return (h1+h2)*100+m2
add_time(900, 60)

1000

In [28]:
import itertools
def get_day_iternery(locations):
    # Initialize a list to store the feasible orders of the locations.
    feasible_orders = []

    # Iterate over all combinations of the locations.
    for order in itertools.permutations(locations):
        cur_time = add_time(order[0]['opening_time'] ,order[0]['duration'])
        order_schedule = [[order[0]['id'],order[0]['opening_time'],cur_time]]  # Store the current order's schedule

        for i in range(1, len(order)):
            travel_time = time_between_locations(order[i - 1]['location'], order[i]['location'])*60
            travel_time = (round(travel_time)//10)*10 +10
            cur_time = add_time(cur_time,travel_time)
            cur_time = max(order[i]['opening_time'], cur_time)
            exit_time = add_time(cur_time ,order[i]['duration'])

            if exit_time > order[i]['closing_time']:
                break

            # Append the id, start time, and end time to the order_schedule
            order_schedule.append([order[i]['id'], cur_time, exit_time])
            cur_time = exit_time

        if len(order_schedule)==len(order):
            feasible_orders.append(order_schedule)

    # If there are no feasible orders, return None.
    if not feasible_orders:
        return None
    else:
        best_idx = 0
        best_time = 100000
        for i in range(len(feasible_orders)):
            tot_time = feasible_orders[i][-1][2]-feasible_orders[i][0][1]
            if tot_time<=best_time:
                best_idx = i
                best_time = tot_time
        return feasible_orders[best_idx]

locations = [{'id': 931, 'opening_time': 1000, 'closing_time': 1800, 'duration': 120, 'location': [17.4292172, 78.32431869999999]}, {'id': 933, 'opening_time': 1000, 'closing_time': 1800, 'duration': 240, 'location': [19.2314895, 72.8075257]}, {'id': 1010, 'opening_time': 1100, 'closing_time': 2300, 'duration': 120, 'location': [19.1353022, 72.83233539999999]}, {'id': 1001, 'opening_time': 1100, 'closing_time': 2300, 'duration': 180, 'location': [19.0999411, 72.9159104]}]

get_day_iternery(locations)

In [1]:
import sqlite3
from scheduler.constants import *
import json

DB_ADDRESS = "../Data/tws2.db"

def open247(day_results):
    for day in day_results:
        if day[3] == 1:
            return 0
    return 1

def get_timing(id):
    query = f"""
        SELECT * from OperationHours WHERE attraction_id = {id}
    """
    conn = sqlite3.connect(DB_ADDRESS)
    cursor = conn.cursor()
    cursor.execute(query)
    attractions_results = cursor.fetchall()
    if len(attractions_results)==1:
        attractions_results = STD_TIMING
    if open247(attractions_results):
        attractions_results = ALL_TIMING

    timing_data = {}
    
    for index, day_result in enumerate(attractions_results):
        day_dict = {}
        
        day_dict['day'] = DAY_MAP[day_result[2]]  # Get the day name from the list
        day_dict['open'] = 'OPEN' if bool(day_result[3]) else 'CLOSE'  # Convert the 'open' value to a boolean
        day_dict['opening_time'] = int(day_result[4])  # Get the opening time
        day_dict['closing_time'] = day_result[5]  # Get the closing time
        
        timing_data[index]=day_dict
    
    return json.dumps(timing_data)

    
get_timing(1000)

'{"0": {"day": "MON", "open": "OPEN", "opening_time": 1000, "closing_time": "1800"}, "1": {"day": "TUE", "open": "OPEN", "opening_time": 1000, "closing_time": "1800"}, "2": {"day": "WED", "open": "OPEN", "opening_time": 1000, "closing_time": "1800"}, "3": {"day": "THU", "open": "OPEN", "opening_time": 1000, "closing_time": "1800"}, "4": {"day": "FRI", "open": "OPEN", "opening_time": 1000, "closing_time": "1800"}, "5": {"day": "SAT", "open": "OPEN", "opening_time": 1000, "closing_time": "1800"}, "6": {"day": "SUN", "open": "OPEN", "opening_time": 1000, "closing_time": "1800"}}'

In [4]:
def get_duration(id):
    query = f"""
        SELECT duration from Attractions WHERE id = {id}
    """
    conn = sqlite3.connect(DB_ADDRESS)
    cursor = conn.cursor()
    cursor.execute(query)
    attractions_results = cursor.fetchall()[0][0]
    return attractions_results

get_duration(324)

1

In [8]:
def get_city_idx(name):
    # Connect to the SQLite database
    conn = sqlite3.connect('../Data/tws.db')
    cursor = conn.cursor()

    query = f"""
        SELECT id FROM Cities WHERE name= '{name}'
    """
    # Fetch data from the database
    cursor.execute(query)
    data = cursor.fetchall()

    # Close the database connection
    conn.close()

    return data[0][0] 

get_city_idx('Mumbai')

0

In [8]:
import requests

# Set the latitude and longitude coordinates
latitude,longitude = [40.7115776,-74.0133362]

latitude,longitude = [19.0168625,72.83045310000001]

# Set the API endpoint
request_url = f"https://nominatim.openstreetmap.org/reverse?lat={latitude}&lon={longitude}"

# Make the request
response = requests.get(request_url)
print(response.content)


b'<?xml version="1.0" encoding="UTF-8" ?>\n<reversegeocode timestamp="Sat, 02 Sep 2023 21:06:00 +00:00" attribution="Data \xc2\xa9 OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright" querystring="lat=19.0168625&amp;lon=72.83045310000001&amp;format=xml"><result place_id="164234648" osm_type="way" osm_id="236614961" ref="\xe0\xa4\xb8\xe0\xa4\xbf\xe0\xa4\xa6\xe0\xa5\x8d\xe0\xa4\xa7\xe0\xa4\xbf\xe0\xa4\xb5\xe0\xa4\xbf\xe0\xa4\xa8\xe0\xa4\xbe\xe0\xa4\xaf\xe0\xa4\x95 \xe0\xa4\xae\xe0\xa4\x82\xe0\xa4\xa6\xe0\xa4\xbf\xe0\xa4\xb0" lat="19.0169869" lon="72.8302725" boundingbox="19.0165709,19.0175282,72.8299696,72.8309914" place_rank="30" address_rank="30">\xe0\xa4\xb8\xe0\xa4\xbf\xe0\xa4\xa6\xe0\xa5\x8d\xe0\xa4\xa7\xe0\xa4\xbf\xe0\xa4\xb5\xe0\xa4\xbf\xe0\xa4\xa8\xe0\xa4\xbe\xe0\xa4\xaf\xe0\xa4\x95 \xe0\xa4\xae\xe0\xa4\x82\xe0\xa4\xa6\xe0\xa4\xbf\xe0\xa4\xb0, Swatantrya Veer Savarkar Marg, G/N Ward, Zone 2, Mumbai, Mumbai City District, Maharashtra, 400025, India</result><addressparts

AttributeError: 'dict' object has no attribute 'append'

In [19]:
# Import API class from pexels_api package
from pexels_api import API


# Type your Pexels API
PEXELS_API_KEY = 'ARe2hWfm1t82LI4DNHY5jIBokogficcSj5N5NLq82nE7Mxh6M0TExRP1'
# Create API object
api = API(PEXELS_API_KEY)
# Search five 'kitten' photos
api.search('Gateway of India,Mumbai', page=1, results_per_page=5)
# Get photo entries
photos = api.get_entries()

In [20]:
print(len(photos))
print(photos)

for ele in photos:
    print(ele.original)

5
[<pexels_api.tools.photo.Photo object at 0x117194580>, <pexels_api.tools.photo.Photo object at 0x117344280>, <pexels_api.tools.photo.Photo object at 0x117149370>, <pexels_api.tools.photo.Photo object at 0x117149280>, <pexels_api.tools.photo.Photo object at 0x117136dc0>]
https://images.pexels.com/photos/1371168/pexels-photo-1371168.jpeg
https://images.pexels.com/photos/2815093/pexels-photo-2815093.jpeg
https://images.pexels.com/photos/6481558/pexels-photo-6481558.jpeg
https://images.pexels.com/photos/13091771/pexels-photo-13091771.jpeg
https://images.pexels.com/photos/6067512/pexels-photo-6067512.jpeg


In [25]:
from datetime import datetime

# format_time = lambda time_str: datetime.strptime(str(time_str).zfill(4), "%H%M").strftime("%H:%M")

def format_time(tt):
    tt=str(tt).zfill(4)
    h,m = tt[:2],tt[2:]
    return h+":"+m


In [29]:
def get_photos(id):
    url_list = [
        'https://images.pexels.com/photos/1371168/pexels-photo-1371168.jpeg',
        'https://images.pexels.com/photos/2815093/pexels-photo-2815093.jpeg',
        'https://images.pexels.com/photos/6481558/pexels-photo-6481558.jpeg',
        'https://images.pexels.com/photos/13091771/pexels-photo-13091771.jpeg',
        'https://images.pexels.com/photos/6067512/pexels-photo-6067512.jpeg'
    ]
    return url_list

get_photos(987)

['https://images.pexels.com/photos/1371168/pexels-photo-1371168.jpeg',
 'https://images.pexels.com/photos/2815093/pexels-photo-2815093.jpeg',
 'https://images.pexels.com/photos/6481558/pexels-photo-6481558.jpeg',
 'https://images.pexels.com/photos/13091771/pexels-photo-13091771.jpeg',
 'https://images.pexels.com/photos/6067512/pexels-photo-6067512.jpeg']